In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# set path to chromedriver and set the browser to chrome
executable_path = {'executable_path': 'C:\Program Files (x86)\Chromedriver\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# declare a URL variable to visit the nasa news page and visit that URL.  Pause for 2 seconds to allow the page to fully load.
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
time.sleep(2)

In [4]:
#Scrape the data into BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#Find the title and text for the most recent article
title = soup.find(class_='content_title')
text = soup.find(class_='rollover_description_inner')
news_title = title.text
news_text = text.text

In [5]:
# Visit the following URL, scrape the data into BS to be discected.  
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.visit(url)

# Declare a partial URL to be used later
beginning_url = 'https://www.jpl.nasa.gov'

In [6]:
# Design an XPATH selector to grab the featured image from the page
xpath = '//a[@class="button fancybox"]'
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [7]:
# Scrape the browser into soup and use soup to find the full resolution image of mars
# Save the image url to a variable called `img_url`
time.sleep(2)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
img_url = soup.find("img", class_="fancybox-image")["src"]
img_url

'/spaceimages/images/mediumsize/PIA14712_ip.jpg'

In [8]:
# Use the partial URL from earlier to combine it with the other partial URL.  
featured_image_url = beginning_url + img_url
featured_image_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14712_ip.jpg


In [9]:
# Declare our new URL as twitter, visit the page, and scrape the data into BS.
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
# Find the most recent tweet and store it in a variable
tweets = soup.find_all("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
for tweet in tweets:
    sol = tweet.text[0:3]
    if sol =

'Sol 2085 (2018-06-18), Sunny, high -24C/-11F, low -57C/-70F, pressure at 7.72 hPa, daylight 05:18-17:22'

In [23]:
atweet = soup.find_all("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
for tweet in atweet:
    sol = tweet.text[0:3]
    if sol == "Sol":
        weather = tweet.text
        break
weather

'Sol 2085 (2018-06-18), Sunny, high -24C/-11F, low -57C/-70F, pressure at 7.72 hPa, daylight 05:18-17:22'

In [ ]:
#Declare another new URL for mars facts and read the tables on the page using pandas.
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

In [ ]:
# Store the data in a dataframe for later
facts_df = tables[0]
facts_df.columns = ["Description", "Value"]
facts_df.set_index('Description', inplace=True)
facts_df.to_html('table.html')
facts_df

In [ ]:
html_table = facts_df.to_html()
html_table = html_table.replace('\n', '')
html_table

In [ ]:
# Declare another URL to get enhanced images of the 4 hemispheres of mars.  Scrape the data into BS to get links to the enhanced images.
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
base_url = "https://astrogeology.usgs.gov"

In [ ]:
# Locate the individual URL for each hemisphere and store them in the img_url list to be iterated through.
hemispheres = soup.find_all("div", class_="item")
img_url = []
for h in hemispheres:
    img_url.append(h.a["href"])

In [ ]:
# loop through the img_url list, go to each link, scrape the data into soup, and locate the title and hyperlink for each 
# enhanced image.  Then store the information as a dictionary.

hemisphere_image_urls = []

for hemi in img_url:
    url = base_url + hemi
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all(class_="downloads")
    link = links[0].ul
    interum_links = []
    header = soup.find(class_="title")
    hemi_title = header.text
    print(hemi_title)
    for x in link:
        interum_links.append(x)
    full_link = interum_links[1].a["href"]
    print(full_link)
    print("-"*40)
    hemisphere_image_urls.append({"title": hemi_title, "img_url": full_link})

In [ ]:
combined_dictionary = {"news_title": news_title,
                      "news_text": news_text,
                      "featured_image_url": featured_image_url,
                      "mars_weather": mars_weather,
                      "hemisphere_image_urls": hemisphere_image_urls,
                      "facts_df": facts_df}